In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Hotel reservations") \
  .getOrCreate()

24/10/22 23:42:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
data = spark.read.csv("data/hotel-reservations.csv", header=True, inferSchema=True)

data.printSchema()

root
 |-- Booking_ID: string (nullable = true)
 |-- no_of_adults: integer (nullable = true)
 |-- no_of_children: integer (nullable = true)
 |-- no_of_weekend_nights: integer (nullable = true)
 |-- no_of_week_nights: integer (nullable = true)
 |-- type_of_meal_plan: string (nullable = true)
 |-- required_car_parking_space: integer (nullable = true)
 |-- room_type_reserved: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_year: integer (nullable = true)
 |-- arrival_month: integer (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- market_segment_type: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- no_of_previous_cancellations: integer (nullable = true)
 |-- no_of_previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- no_of_special_requests: integer (nullable = true)
 |-- booking_status: string (nullable = true)



In [3]:
data.count()

36275

In [4]:
data.show()

+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+------------------+----------------------+--------------+
|Booking_ID|no_of_adults|no_of_children|no_of_weekend_nights|no_of_week_nights|type_of_meal_plan|required_car_parking_space|room_type_reserved|lead_time|arrival_year|arrival_month|arrival_date|market_segment_type|repeated_guest|no_of_previous_cancellations|no_of_previous_bookings_not_canceled|avg_price_per_room|no_of_special_requests|booking_status|
+----------+------------+--------------+--------------------+-----------------+-----------------+--------------------------+------------------+---------+------------+-------------+------------+-------------------+--------------+----------------------------+------------------------------------+--

In [5]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Train size: ", train_data.count())
print("Test size: ", test_data.count())

Train size:  29040
Test size:  7235


In [6]:
pipeline_stages = []

In [7]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder 

categorical_cols = ['type_of_meal_plan', 'room_type_reserved', 'arrival_month', 'market_segment_type']

for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol= col + '_index')
    print(f'StringIndexer {string_indexer.getInputCol()} -> {string_indexer.getOutputCol()}')
    
    encoder = OneHotEncoder(inputCol=string_indexer.getOutputCol(), outputCol=col + '_vec', dropLast=False)
    print(f'OneHotEncoder {encoder.getInputCol()} -> {encoder.getOutputCol()}')
    print()
    
    pipeline_stages += [string_indexer, encoder]


StringIndexer type_of_meal_plan -> type_of_meal_plan_index
OneHotEncoder type_of_meal_plan_index -> type_of_meal_plan_vec

StringIndexer room_type_reserved -> room_type_reserved_index
OneHotEncoder room_type_reserved_index -> room_type_reserved_vec

StringIndexer arrival_month -> arrival_month_index
OneHotEncoder arrival_month_index -> arrival_month_vec

StringIndexer market_segment_type -> market_segment_type_index
OneHotEncoder market_segment_type_index -> market_segment_type_vec



In [8]:
pipeline_stages += [StringIndexer(inputCol='booking_status', outputCol= 'booking_status_index')]

In [9]:
encoded_categorical_cols = [col + "_vec" for col in categorical_cols]
encoded_categorical_cols

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec']

In [10]:
numeric_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
               'required_car_parking_space', 'lead_time',
               'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
               'avg_price_per_room', 'no_of_special_requests'] 

In [11]:
input_columns = encoded_categorical_cols + numeric_cols
input_columns

['type_of_meal_plan_vec',
 'room_type_reserved_vec',
 'arrival_month_vec',
 'market_segment_type_vec',
 'no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'required_car_parking_space',
 'lead_time',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [12]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=input_columns, outputCol='features')

pipeline_stages.append(assembler)

In [13]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol='features', labelCol='booking_status_index')

pipeline_stages.append(dtc)

In [14]:
pipeline_stages

[StringIndexer_267a5074b49c,
 OneHotEncoder_e082716042f6,
 StringIndexer_b460a4af6177,
 OneHotEncoder_669c85e4c123,
 StringIndexer_ae3ed6744ffa,
 OneHotEncoder_5e7a88b2310b,
 StringIndexer_ed21e8081d5b,
 OneHotEncoder_6325d1230b8b,
 StringIndexer_51875a86822c,
 VectorAssembler_07542767d97d,
 DecisionTreeClassifier_b921e1675895]

In [15]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=pipeline_stages)

In [16]:
model = pipeline.fit(train_data)

In [17]:
predictions = model.transform(test_data)
predictions.select('features', 'booking_status_index', 'prediction').show(10)

+--------------------+--------------------+----------+
|            features|booking_status_index|prediction|
+--------------------+--------------------+----------+
|(39,[0,4,21,23,28...|                 1.0|       0.0|
|(39,[0,4,11,23,28...|                 0.0|       0.0|
|(39,[0,4,17,24,28...|                 0.0|       0.0|
|(39,[0,4,16,23,28...|                 1.0|       0.0|
|(39,[0,4,11,24,28...|                 0.0|       0.0|
|(39,[0,4,11,24,28...|                 0.0|       0.0|
|(39,[2,4,12,24,28...|                 0.0|       0.0|
|(39,[1,4,18,23,28...|                 0.0|       0.0|
|(39,[0,4,15,24,28...|                 0.0|       1.0|
|(39,[0,4,16,24,28...|                 0.0|       0.0|
+--------------------+--------------------+----------+
only showing top 10 rows



24/10/22 23:42:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

accuracty_evaluator = MulticlassClassificationEvaluator(labelCol='booking_status_index', predictionCol='prediction', metricName='accuracy')
accuracy = accuracty_evaluator.evaluate(predictions) * 100
print(f'Accuracy = {accuracy:.2f}%')

Accuracy = 82.21%


In [19]:
precision_evaluator = MulticlassClassificationEvaluator(labelCol='booking_status_index', predictionCol='prediction', metricName='precisionByLabel')
precision = precision_evaluator.evaluate(predictions) * 100
print(f'Precision = {precision:.2f}%')

Precision = 85.51%


In [20]:
recall_evaluator = MulticlassClassificationEvaluator(labelCol='booking_status_index', predictionCol='prediction', metricName='recallByLabel')
recall = recall_evaluator.evaluate(predictions) * 100
print(f'Recall = {recall:.2f}%')

Recall = 88.45%
